In [8]:
import tensorflow as tf
import numpy as np

#with tf.Session() as sess:
#    new_saver = tf.train.import_meta_graph('model_save.ckpt.meta')
#    new_saver.restore(sess, tf.train.latest_checkpoint('./'))

x_data = np.loadtxt('./inputData.txt', delimiter = ',', unpack = True, dtype = 'float32')
x_data = np.transpose(x_data)
y_data = np.loadtxt('./Label.txt', delimiter = ',', unpack = True, dtype = 'float32')
y_data = np.transpose(y_data)

idx_shuffle = np.random.permutation(len(x_data))
x_data = x_data[idx_shuffle]
y_data = y_data[idx_shuffle]

X = tf.placeholder(tf.float32, name = 'X') # 1 x 7*3, (x, y, z)
Y = tf.placeholder(tf.float32, name = 'Y') # 1 x 2, softmax 

w1 = tf.Variable(tf.random_normal([21, 256], stddev = .1), name='w1')
b1 = tf.Variable(tf.zeros([256]), name='b1')

L1 = tf.add(tf.matmul(X, w1), b1)
L1_act = tf.nn.relu(L1)
                 
w2 = tf.Variable(tf.random_normal([256, 256], stddev = .1), name='w2')
b2 = tf.Variable(tf.zeros([256]), name='b2')

L2 = tf.add(tf.matmul(L1_act, w2), b2)
L2_act = tf.nn.relu(L2)
      
w3 = tf.Variable(tf.random_normal([256, 2], stddev = .1), name='w3')
b3 = tf.Variable(tf.zeros([2]), name='b3')
         
m = tf.add(tf.matmul(L2_act, w3), b3)
model = tf.nn.softmax(m, name = 'model')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001)
train_op = optimizer.minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for (step) in range(10000):
    sess.run(train_op, feed_dict = {X:x_data[0:700], Y:y_data[0:700]})
    
    if (step) % 2000 == 0:
        c = sess.run(cost, feed_dict = {X:x_data[0:700], Y:y_data[0:700]})
        c_test = sess.run(cost, feed_dict = {X:x_data[700:], Y:y_data[700:]})
        #l1 = sess.run(model, feed_dict =  {X:x_data[0:10]}) 
        print(step, "   ", c, "   ", c_test)
        #print(l1)


#mod = sess.run(model, feed_dict = {X:x_data[100:], Y:y_data[100:]})
#mod = np.argmax(mod,1)
#lab = np.argmax(y_data[100:], 1)

correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

print('Accuracy:', sess.run(accuracy, feed_dict = {X:x_data[700:], Y:y_data[700:]}))

saver = tf.train.Saver()
save_path = saver.save(sess, "./model_save.ckpt")

0     0.6981994     0.68984795
2000     0.3746963     0.3921768
4000     0.36293918     0.39327973
6000     0.36062264     0.39266637
8000     0.3604594     0.3922846
Accuracy: 0.9122807
